# 1-1 - Regex

*Sébastien Meyer, Marie Tcheng*

Dans cet atelier, vous allez écrire des fonctions pour segmenter un texte juridique en vous basant sur les formats habituels d'écriture de ces textes.

Voici un exemple d'un tel texte : https://www.georisques.gouv.fr/webappReport/ws/installations/document/cf8c26fa8e9248118f91dd444a11b763

Dans un texte juridique de type "arrêté préfectoral", on retrouve habituellement la structure suivante : 

- En-tête : contient éventuellement la devise nationale, le titre du l'arrêté, l'autorité qui prend l'arrêté...

Exemple : 

```
ARRETE ARS GRAND EST n 2025-0182 du 13 janvier 2025
Portant fixation du bilan quantitatif de l'offre de soins pour la période de dépôt des demandes d'autorisation des activités de soins ouverte du 1er février 2025 au 1er avril 2025 pour la région Grand Est
La Directrice Générale de l'Agence Régionale de Santé Grand Est
```

- Visas : ce sont les textes visés par cet arrêté qui s'appliquent en premier lieu

Exemple :

```
- VU le Code de la santé publique, et notamment ses articles L 6122-9 et R 6122-30.
- VU le décret du 21 mai 2024 portant nomination de Mme Christelle RATIGNIER CARBONNEIL en qualité de la directrice Générale de l'Agence Régionale de Santé Grand Est.
```

- Motifs : ce sont les raisons pour lesquelles l'arrêté a été pris

Exemple : 

```
CONSIDÉRANT les risques sur certains sentiers de randonnée pédestre situés sur le domaine forestier géré par l'ONF de La Réunion à la suite du passage du cyclone Garance ;
CONSIDÉRANT qu'il y a lieu d'assurer la sécurité publique ;
```

- Titre : un "Arrête" en lettres capitales symbolise le début des prescriptions

Exemple : 

```
A R R E T E 
```

- Corps de texte : ce sont des articles qui se suivent, chaque article peut contenir des sous-articles et/ou des alinéas. Un alinéa définit des prescriptions, il s'agit en règle générale d'un paragraphe bien qu'en réalité ce soit plus subtil 

Exemple : 

```
Article 1er

La circulation de tous les véhicules à moteur est interdite sur :

- l'autoroute A1, dans les deux sens de circulation entre la jonction des échangeurs A1/A21 à Dourges et le tronc commun A1/22 à Lille ;

Article 2

Sur l'A1, l'ensemble des bretelles d'insertion sur l'autoroute A1 sont fermées entre la jonction des échangeurs A1/A21 à Dourges et le tronc commun A1/22 à Lille dans le sens Paris vers Lille. La bretelle d'insertion est fermée à l'échangeur 20 (Lesquin) dans le sens Lille vers Paris.
```

- Signature : il y a toujours une signature de l'autorité compétente qui prend l'arrêté 

Exemple : 

```
Pour le préfet de zone et par délégation, Le préfet délégué pour la défense et la sécurité
Vincent LASOGUEY
```

- Annexes : il peut s'agir de tableaux, de plans, etc.

Exemple : 

```
ANNEXE 1 : modèle du bulletin de vote 
Modèle de bulletin de vote
```

**NB** : Dans les visas et le corps de texte, on peut retrouver des citations vers d'autres textes juridiques. Ainsi, il est important de pouvoir faire un lien entre un texte juridique et les textes sur lesquels il s'appuie.

## Imports initiaux

Vous ne devez pas modifier cette partie du code. Il s'agit du nécessaire pour que vous puissiez remplir l'exercice.

In [ ]:
import re

## Introduction des concepts utilisés

Pour réaliser le parsing de structures on utilise des expressions régulières ou regex. On appelle "contenu matché" le contenu d'une phrase qui correpond à la recherche regex. Un moteur regex réalise une comparaison entre une phrase d'entrée et un *pattern*. 

Sur Python les regex sont implémentées dans le module *re* disponible par défaut. On écrit les regex dans une raw string (ex : r"votre_regex") et on l'utilise après compilation. On peut utiliser plusieurs méthodes pour détecter des patterns dans une phrase comme la méthode *search* qui recherche si le pattern est présent au moins une fois dans la phrase. 

Exemple : 

```python
example = re.search(
    # Le pattern
    r"contenu",
    # La phrase dans laquelle chercher le pattern
    "contenu entier",
    # Les options éventuelles (ex : ignorer la casse
    re.IGNORECASE,
)
```

Cette commande renvoie *None* s'il n'y a aucune match et sinon un objet de type *re.Match*.

Nous décrivons ici quelques options classiques pour les regex. Vous êtes libres d'utiliser internet pour vous renseigner sur comment les utiliser. 

- "[contenu]" : un des éléments de ce groupe

Exemple : Cette regex peut détecter les n minuscules ou majuscules

```python
re.search(r"[Nn]", "nnuméro")
```

- "(contenu)" : sépare des groupes

- "|" : il s'agit du "ou" regex qui matche si un des éléments est trouvé

Exemple : Cette regex peut détecter si le mot article ou chapitre est présent

```python
re.search(r"(article|chapitre)", "chapitre 3") 
```

- "\s" : les espaces

Exemple : 

```python
re.search(r"article\s3", "article 3")
```

- "\d" : les chiffres

Exemple : Cette regex détecte s'il y a un chiffre

```python
re.search("\d", "3")
```

- "*" : zéro ou plusieurs éléments

- "+" : un ou plusieurs éléments

Exemple : Cette regex détecte s'il y a un chiffre ou plus

```python
re.search(r"\d+", "123")
```

- "?" : l'élément est optionnel

Exemple : 

```python
re.search(r"article (modifié)? 3", "article 3 article modifié 3")
```

- ".*" : tout le contenu

Exemple : 

```python
re.search(r"name .* is valid", "name ez2984kd is valid")
```

- (?P\<groupe\>...) : définit une balise qui récupère un contenu et l'extrait

Exemple : 

```python
re.search(r"article (?P<number>\d+)", "article 3")
```

- "$" : il doit s'agir de la fin de la phrase

Exemple : 

```python
re.search(r"article$", "article")
```

- "{n, p}" : dénombrement des éléments dans cette gamme

Exemple : 

```python
re.search(r"(\d){1, 2}", "34")
```

- "!(contenu)" : le groupe suivant ne doit pas être dans la phrase. A utiliser avec parcimonie car peut être coûteux et difficile à maîtriser

Exemple : 

```python
re.search("!(article)\d", "chapitre 3")
```

**NB** : Certains caractères qui sont des commandes doivent donc être échappés pour être utilisés 

Exemple : 

```python
re.search(r"phrase\.", "phrase.")
```

## Exercice 1 - détection des codes

Dans la législation française, il existe une liste finie de codes qui regroupent un ensemble de concepts législatifs et réglementaires. 

Nous sommes intéressés par reconnaître les codes suivants : 
- code du travail
- code de l'environnement
- code minier
- code forestier 

Construisez un détecteur de code à partir de regex. 

In [ ]:
code_pattern = r"""VOTRE CODE ICI"""

In [ ]:
result = re.search(code_pattern, "Bla bla code de l'environnement")
assert result

## Exercice 2 - détection des décrets

Il existe plusieurs types de décrets qui sont donc numérotés avec des chiffres éventuellemen séparés par des tirets.

*Exemple* : 

```
décret n°2005-635 du 30 mai 2005
```

Construisez une regex pour détecter les décrets. Le numéro doit être extrait dans une balise *identifier*. 

In [ ]:
decret_pattern = r"""VOTRE CODE ICI"""

In [ ]:
result = re.search(decret_pattern, "Bla bla décret n°2005-635 du 30 mai 2005 relatif à")
assert result
assert result.group("identifier") == "2005-635"

## Exercice 3 - détection des arrêtés

Il existe de nombreux types d'arrêtés en France : ministériel, préfectoral, sans dénomination particulière.

Exemples : 

```
arrêté du 2 février 1998
arrêté ministériel du 2 février 1998
arrêtés préfectoraux n° 5213 du 28 octobre 1988 et n° 1636 du 24/03/95
```

Construisez une regex pour détecter les arrêtés au singulier.

Construisez une regex pour détecter les arrêtés au pluriel avec leur numérotation extraite dans une balise *identifierX* avec X le numéro de l'identifiant.

Pour les arrêtés au singulier

In [ ]:
arrete_pattern = r"""VOTRE CODE ICI"""

In [ ]:
result = re.search(arrete_pattern, "arrêté du 2 février 1998")
assert result

In [ ]:
result = re.search("arrêté ministériel du 2 février 1998")

Et pour les arrêtés au pluriel

In [ ]:
arrete_multiple_pattern = r"""VOTRE CODE ICI"""

In [ ]:
result = re.search(arrete_multiple_pattern, "arrêtés préfectoraux n° 5213 du 28 octobre 1988 et n° 1636 du 24/03/95")
assert result
assert result.group("identifier1") == "5213"
assert result.groupe("identifier2") == "1636"

## Exercice 4 - détection des articles

Il existe plusieurs formats de titres selon les arrêtés. Les parties de code suivantes permettent de petit à petit construire un module qui détecte les différents titres tout en évitant de mal détecter d'autres éléments qui n'en sont pas. 

On souhaite détecter les cas suivants :

```
ARTICLE 1

TITRE I - Premier titre
Chapitre A - Premier chapitre
ARTICLE 1.1.1 Premier article

# Numérotation avant le titre 
1.1.1 Article
# Numérotation séparée par un tiret
Article 2-1 Article
# Numérotation romaine
II.1.1 - Sous-titre
# Annexe
ANNEXÉ 3 - Attestation du Prestataire
```

Construisez une regex pour ce cas. Vous devez extraire les éléments *section_type*, *number*, *text*. 

In [ ]:
title_pattern = r"""VOTRE CODE ICI"""

In [ ]:
result = re.search(title_pattern, "ARTICLE 1")
assert result
assert result.group("section_type") == "ARTICLE"
assert result.group("number") == "1"

In [ ]:
result = re.search(title_pattern, "TITRE I - Premier titre")
assert result
assert result.group("section_type") == "TITRE"
assert result.group("number") == "I"
assert result.group("text") == "Premier titre"

In [ ]:
result = re.search(title_pattern, "Chapitre A - Premier chapitre")
assert result
assert result.group("section_type") == "Chapitre"
assert result.group("number") == "A"
assert result.group("text") == "Premier chapitre"

In [ ]:
result = re.search(title_pattern, "ARTICLE 1.1.1 Premier article")
assert result
assert result.group("section_type") == "ARTICLE"
assert result.group("number") == "1.1.1"
assert result.group("text") == "Premier article"

In [ ]:
result = re.search(title_pattern, "1.1.1 Article")
assert result
assert result.group("section_type") == "Article"
assert result.group("number") == "1.1.1"

In [ ]:
result = re.search(title_pattern, "Article 2-1 Article")
assert result
assert result.group("section_type") == "Article"
assert result.group("number") == "2-1"
assert result.group("text") == "Article"

In [ ]:
result = re.search(title_pattern, "II.1.1 - Sous-titre")
assert result
assert result.group("number") == "II.1.1"
assert result.group("text") == "Sous-titre"

In [ ]:
result = re.search(title_pattern, "ANNEXÉ 3 - Attestation du Prestataire")
assert result
assert result.group("section_type") == "ANNEXÉ"
assert result.group("number") == "3"
assert result.group("text") == "Attestation du Prestataire"

-----